In [ ]:
from scipy.optimize import dual_annealing
import numpy as np
from calc_cover_time import *
def optimize_multi_drones(params, drone_positions, missile_trajs, true_target_corners, smoke_radius, t_list, debug=False):
    params = np.array(params)

    init_params = params.reshape(5, 8)

    bounds = []
    for i in range(5):
        for j in range(8):
            p = init_params[i, j]
            if j == 0:
                lb = max(70, p * 0.9)
                ub = min(140, p * 1.1)
            elif j == 1:
                lb = max(0, p - abs(p) * 0.1)
                ub = min(2 * np.pi, p + abs(p) * 0.1)
            elif j in [2, 3, 4]:
                lb = max(0, p * 0.9)
                ub = min(60, p * 1.1)
            else:
                lb = max(0.1, p * 0.9)
                ub = min(20, p * 1.1)
            bounds.append((lb, ub))

    def objective(x):
        try:
            x = np.array(x).reshape(5, 8)
            cover_times = []
            covered_all = np.zeros(len(t_list), dtype=bool)
            for i in range(5):
                speed = x[i, 0]
                angle = x[i, 1]
                throw_times = np.sort(x[i, 2:5])
                burst_delays = x[i, 5:8]
                smoke_centers = get_smoke_center_multi(
                    drone_positions[i], angle, speed, throw_times, burst_delays, t_list
                )
                cover_time_i = get_missile_cover_time_multi_corners(
                    smoke_centers, missile_trajs[i], true_target_corners, smoke_radius, t_list, debug=False
                )
                covered_all |= cover_time_i
                cover_times.append(cover_time_i)
            total_cover_time = np.sum(covered_all) * (t_list[1] - t_list[0])
            return -total_cover_time
        except AssertionError:
            return 1e6

    result = dual_annealing(objective, bounds, maxiter=3000, seed=42, initial_temp=10000)
    best_x = result.x.reshape(5, 8)
    cover_times = []
    print("优化后参数：")
    print(best_x)
    print("详细遮蔽信息：")
    for i in range(5):
        speed = best_x[i, 0]
        angle = best_x[i, 1]
        throw_times = np.sort(best_x[i, 2:5])
        burst_delays = best_x[i, 5:8]
        smoke_centers = get_smoke_center_multi(
            drone_positions[i], angle, speed, throw_times, burst_delays, t_list
        )
        print(f"\n无人机{i+1}:")
        cover_time = get_missile_cover_time_multi_corners(
            smoke_centers, missile_trajs[i], true_target_corners, smoke_radius, t_list, debug=True
        )
        cover_times.append(cover_time)
    cover_times = np.array(cover_times)
    print(f"\n所有导弹被遮蔽时间分别为: {cover_times}")
    print(f"方差: {np.var(cover_times):.4f}")
    print(f"最大遮蔽总时间: {np.sum(cover_times):.2f}s")
    return np.sum(cover_times), cover_times, np.var(cover_times)